# Commodity Risk Management
### Asymmetric Volatility Forecasting & Dynamic VaR
*GJR-GARCH Approach*

---

> **The Core Observation:** ARCH/GARCH models are built on the idea that volatility spikes suddenly and persists before reverting to its long-term mean. This creates a phenomenon known as *volatility clustering*.

> **Mechanics:** In the GARCH framework:
> * $u_{t-1}^2$ represents the **shock** generating volatility.
> * The $\beta$ (Beta) coefficient captures the **persistence** of volatility after a shock.

*(Extensions included: Exponential GARCH, asymmetric models, regime-switching, etc.)*

In [ ]:
%matplotlib inline

## 01. Data Extraction & Preparation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

In [ ]:
data = yf.download('BNO', start = "2015-01-01", end = '2025-11-27')

data.columns = data.columns.droplevel(1)

data

In [ ]:
prices = data.Close

returns = np.log(prices/prices.shift(1))

data['Returns'] = returns

data

In [ ]:
data = data.dropna()

data

## 02. Descriptive Analysis & Visualization

In [ ]:
returns.plot(figsize=(16,9), color = 'red')
plt.xlabel('Date')
plt.ylabel('Returns')
plt.grid(True)
plt.show()

> **Visual inspection of the Brent log-return series reveals four key stylized facts:**
>
> 1. **Stationarity & Mean Reversion 🔄**
>    Unlike the raw price series which exhibits a stochastic trend, the return series oscillates around a constant mean close to zero. This stationarity is crucial for modeling.
>
> 2. **Volatility Clustering (Heteroscedasticity) 〰️**
>    The plot illustrates Mandelbrot's observation: *"Large changes tend to be followed by large changes."* We see distinct regimes of calm (2017) vs. turbulence (2020, 2022).
>
> 3. **Impact of Exogenous Shocks 💥**
>    Extreme spikes correspond to real-world events like the **COVID-19 oil crash** (-25% drop) or the 2022 geopolitical tensions.
>
> 4. **Leptokurtosis & "Fat Tails" 🔔**
>    The distribution significantly deviates from a Normal Law. Extreme events happen far more frequently than a Gaussian model would predict.

In [ ]:
(returns ** 2).plot(figsize=(16,9), color = 'red')
plt.xlabel('Date')
plt.ylabel('Squared Returns')
plt.grid(True)
plt.show()

> **📉 Squared Returns ($r_t^2$)**
>
> They are essential because they constitute the best observable proxy for the *unobservable* conditional variance (volatility) of returns at a given time.
>
> *(Since daily returns usually have a mean very close to zero ($\mu \approx 0$), the squared return acts as a direct approximation of the variance.)*
>
> 1. **Visualizing Volatility Clustering 👁️**
>    They allow for a clear visualization of volatility clustering (regrouping of high-variance periods) which might be less obvious in raw returns.
>
> 2. **Model Input (Variance Equation) 📐**
>    They serve as the dependent variable (the input) for the variance equation within the GARCH model structure.
>
> 3. **Statistical Testing (ARCH-LM / Ljung-Box) 🧪**
>    They are the subject of the **Ljung-Box** or **ARCH-LM test**. If $r_t^2$ are significantly autocorrelated, it proves that volatility has temporal dependence and that a GARCH model is statistically necessary.

In [ ]:
import statsmodels
from statsmodels.stats.diagnostic import acorr_ljungbox

In [ ]:
returns_clean = returns.dropna()

In [ ]:
ljung_box = acorr_ljungbox(returns_clean**2., lags = 10, return_df = True)

print(ljung_box.apply(lambda x: x.map('{:.4f}'.format)))

## 03. Conditional Mean Modeling

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
p = 1
q = 1
model = ARIMA(returns,order=(p,0,q))
arma = model.fit()

print(arma.summary())

> **Hypothesis Testing: The Zero-Mean Assumption**
>
> 1. **Statistical Evidence 📉**
>    The sample mean is extremely low (approx. 0.000009). More importantly, the **p-value > 0.05** implies that we fail to reject the null hypothesis ($H_0: \mu = 0$). The mean is statistically indistinguishable from zero.
>
> 2. **Modeling Implication 🎯**
>    Since $\mu \approx 0$, we can simplify the conditional mean equation. The return $r_t$ is approximately equal to the residual (innovation) term:
>
>    $$r_t \approx \epsilon_t$$
>
>    This allows us to focus directly on modeling the variance (GARCH) without fitting a complex ARIMA model for the mean first.

In [ ]:
residuals = arma.resid

residuals_clean = residuals.dropna()

residuals_clean

## 04. GARCH vs. GJR-GARCH Modeling

In [ ]:
from arch import arch_model

am = arch_model(residuals_clean*100, mean = 'Constant', vol = 'Garch', p = 1, q = 1, dist='Normal' )

res_garch = am.fit(disp='off')

print(res_garch.summary())

In [ ]:
gjr = arch_model(residuals_clean*100, mean = 'Constant', vol = 'GARCH', p = 1, o = 1, q = 1, dist = 't')
res_gjr = gjr.fit(disp='off')
print(res_gjr.summary())

### 🏆 Model Selection: Statistical Comparison

| Metric | Target | GARCH(1,1) | GJR-GARCH(1,1) |
| :--- | :---: | :---: | :---: |
| **Log-Likelihood** | (Maximize ↑) | -5864.30 | **-5785.12** |
| **AIC** | (Minimize ↓) | 11736.60 | **11582.25** |
| **BIC** | (Minimize ↓) | 11760.26 | **11617.75** |

> **✅ Conclusion:**
>
> The **GJR-GARCH** model is superior. It exhibits a higher *Log-Likelihood* (better fit) and lower Information Criteria (AIC & BIC), indicating that the inclusion of the asymmetry parameter justifies the slight increase in model complexity.

## 05. Statistical Validation & Model Diagnostics

> **🔍 Diagnostics on Standardized Residuals ($z_t$)**
>
> To validate the GJR-GARCH model, we must inspect the standardized residuals, defined as:
>
> $$z_t = \frac{\epsilon_t}{\sigma_t}$$
>
> 1. **No Autocorrelation (White Noise) 📉**
>    The standardized residuals should behave like white noise. We use the **Ljung-Box Test** to confirm that no linear dependence remains.
>
> 2. **No Remaining ARCH Effects 🧬**
>    We perform the **ARCH-LM Test** on squared standardized residuals ($z_t^2$) to ensure the model has successfully captured all volatility clustering.
>
> 3. **Distribution Fit 🔔**
>    We compare the distribution of $z_t$ against the assumed distribution (Student-t) using a Q-Q Plot.

In [ ]:
residuals_gjr = res_gjr.std_resid

residuals_gjr_squared = residuals_gjr ** 2

ljung_box_gjr = acorr_ljungbox(residuals_gjr_squared, lags = 10, return_df= True)
print(ljung_box_gjr.apply(lambda x: x.map('{:.4f}'.format)))

> **✅ Validation Success: Absence of ARCH Effects**
>
> 1. **Statistical Evidence (p-values > 0.05) 📊**
>    All p-values from the ARCH-LM test exceed the 5% significance level. Therefore, we cannot reject the Null Hypothesis ($H_0$: no ARCH effects).
>
> 2. **Interpretation 🧹**
>    There is no remaining autocorrelation in the squared standardized residuals. The GJR-GARCH model has successfully **"cleaned"** all conditional heteroscedasticity from the data.
>
> 3. **Conclusion 🏁**
>    The residuals are now **homoscedastic** (they behave like white noise regarding variance). The model is statistically valid and robust for forecasting.

## 06. Volatility Forecasting & Value-at-Risk (VaR)

> **🔮 From Volatility to Risk Management: Dynamic VaR**
>
> 1. **The Concept 🛡️**
>    Value-at-Risk (VaR) estimates the maximum potential loss over a specific time horizon at a given confidence level (e.g., 95%). Unlike static methods, our **Dynamic VaR** adjusts daily based on the GJR-GARCH volatility forecast.
>
> 2. **The Formula 📐**
>    Since the mean is negligible ($\mu \approx 0$), the One-Day VaR at significance level $\alpha$ (e.g., 5%) is calculated as:
>
>    $$\text{VaR}_t = \sigma_t \times Q_\alpha$$
>
>    Where:
>    * $\sigma_t$ : Conditional volatility forecast by GJR-GARCH.
>    * $Q_\alpha$ : The quantile of the Student-t distribution (capturing fat tails).
>
> 3. **Interpretation 💡**
>    If the actual return falls below the VaR line, it is considered a **"VaR Breach"** (or exception). This signals an extreme market event that the model predicted with only $\alpha$ probability.

In [ ]:
data_test = yf.download('BNO', start = '2025-11-27', end = '2025-12-05')

prices_test = data_test.Close

returns_test = np.log(prices_test/prices_test.shift(1))

data_test['Returns Test'] = returns_test

data_test

In [ ]:
import scipy.stats as stats

horizon_days = 5
forecasts = res_gjr.forecast(horizon=horizon_days)

pred_vol_series = np.sqrt(forecasts.variance.iloc[-1].values)

real_returns_series = data_test['Returns Test'].dropna()

limit = min(len(real_returns_series), horizon_days)

comparison = pd.DataFrame({
    'Date': real_returns_series.index[:limit],
    'Real Return (%)': real_returns_series.values[:limit] * 100,
    'Forecasted Volatility (%)': pred_vol_series[:limit]
})

nu = res_gjr.params['nu']
t_quantile = stats.t.ppf(0.01, nu)
comparison['VaR 99% (%)'] = comparison['Forecasted Volatility (%)'] * t_quantile

> ### 🏁 Final Test: Forecast vs. Reality (Backtest)
>
> | Date | Real Return (%) | Forecasted Vol (%) | VaR 99% (%) | Status |
> | :--- | :---: | :---: | :---: | :---: |
> | 2025-12-01 | +0.068 % | 1.751 % | **-5.351 %** | ✅ Safe |
> | 2025-12-02 | -1.241 % | 1.764 % | **-5.391 %** | ✅ Safe |
> | 2025-12-03 | +0.508 % | 1.777 % | **-5.421 %** | ✅ Safe |
> | 2025-12-04 | +0.721 % | 1.709 % | **-5.465 %** | ✅ Safe |
>
> *(*Safe* implies that the Real Return > VaR 99%. No breach occurred during this period.)*

In [ ]:
import matplotlib.pyplot as plt

conditional_vol = res_gjr.conditional_volatility 

returns_hist = residuals_clean * 100 

plt.figure(figsize=(14, 7))

plt.plot(returns_hist.index, returns_hist, color='blue', alpha=0.4, lw=1, label='Brent Daily Returns')

plt.plot(conditional_vol.index, conditional_vol, color='red', lw=1.5, label='Estimated Volatility (GJR-GARCH)')
plt.plot(conditional_vol.index, -conditional_vol, color='red', lw=1.5)

plt.title('Brent Volatility Modeling : GJR-GARCH(1,1) Student', fontsize=16)
plt.ylabel('Returns / Volatility (%)', fontsize=12)
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)

plt.show()

## 07. Backtesting & Final Conclusion

> **🏁 Final Test: Forecast vs. Reality**
>
> 1. **Out-of-Sample Backtesting 📅**
>    We compare the model's predictions (*ex-ante*) against the actual market data (*ex-post*) for the most recent trading days. This is the ultimate test of the model's predictive power.
>
> 2. **VaR Breach Check 🚨**
>    We monitor if the **Real Return** falls below the **VaR 99%** threshold.
>    * If *Return* > *VaR*: The risk was correctly covered. ✅
>    * If *Return* < *VaR*: It is a **Breach** (Exception). Too many breaches would invalidate the model.
>
> 3. **Visual Fit 📈**
>    The final plot illustrates the **Vol-Envelope**. The red line (Estimated Volatility) should tightly hug the blue spikes (Returns), expanding during crises and contracting during calm periods.

In [ ]:
comparison['VaR Validation'] = np.where(
    comparison['Real Return (%)'] < comparison['VaR 99% (%)'], 
    '❌ EXCEPTION',  
    '✅ OK'       
)

comparison['Score Sigma'] = comparison['Real Return (%)'] / comparison['Forecasted Volatility (%)']

### 🏁 Backtesting Results: VaR Breach Analysis

| Date | Real Return | Forecast Vol | VaR 99% | Score $\sigma$ | Validation |
| :--- | :---: | :---: | :---: | :---: | :---: |
| 2025-12-01 | +0.069 % | 1.752 % | **-5.351 %** | 0.039 | ✅ OK |
| 2025-12-02 | -1.241 % | 1.765 % | **-5.391 %** | -0.703 | ✅ OK |
| 2025-12-03 | +0.508 % | 1.777 % | **-5.421 %** | 0.331 | ✅ OK |
| 2025-12-04 | +0.722 % | 1.789 % | **-5.465 %** | 0.403 | ✅ OK |

> * **Definition:** Score $\sigma$ measures the return standardized by the forecasted volatility. A score outside [-3, +3] would indicate an extreme event.*

In [ ]:
import scipy.stats as stats
import numpy as np

volatility = res_gjr.conditional_volatility
nu = res_gjr.params['nu']
t_quantile = stats.t.ppf(0.01, nu)
VaR_series = volatility * t_quantile

def kupiec_pof_test(returns, var_series, confidence_level=0.99):
    alpha = 1 - confidence_level
    N = len(returns)
    breaches = returns < var_series
    x = np.sum(breaches)
    p = x / N

    if x == 0:
        print("Aucune brèche observée ! Le modèle est (trop) prudent.")
        return 0, 1.0, x, N
    
    numerator = ((1 - alpha)**(N - x)) * (alpha**x)
    denominator = ((1 - p)**(N - x)) * (p**x)
    
    lr_stat = -2 * np.log(numerator / denominator)
    p_value = 1 - stats.chi2.cdf(lr_stat, df=1)
    
    return lr_stat, p_value, x, N

lr, p_val, failures, total = kupiec_pof_test(residuals_clean * 100, VaR_series, confidence_level=0.99)

> ### ❌ Kupiec Test Diagnosis: Model Validation Failed
>
> *The Kupiec test indicates that the model is **too conservative**. The observed number of exceptions (breaches) is significantly lower than the theoretical expectation for a 99% VaR.*
>
> **📉 The Diagnosis:**
> * **Observed Failures:** 0.62% (vs 1.00% expected).
> * **Implication:** The calculated VaR overestimates the risk. It is "too high" and rarely breached.
> * **Result:** Reject $H_0$ (p-value 0.0315 < 0.05).
>
> **🛠️ Strategic Fixes (Recalibration Roadmap):**
> 1. **Parameter Tuning:** Adjust GJR-GARCH coefficients or the Student-t degrees of freedom (DoF) to better reflect observed losses.
> 2. **VaR Scaling:** Apply a slight multiplicative factor (< 1) to lower the VaR without altering confidence levels, aligning exceptions with theory.
> 3. **Distribution Selection:** Adopt asymmetric distributions (e.g., Skewed Student-t) or semi-parametric methods (Cornish-Fisher) to better capture skewness and fat tails.
> 4. **Iterative Backtesting:** Re-calculate and test iteratively until the model satisfies the Kupiec condition (p-value > 0.05).